In [1]:
# =========================
# SFM – Minimal/Debug Tuning-Pipeline (analog ET/EN/LGBM)
# =========================
# - USE_DYNAMIC_FI_PIPELINE = False  -> "Full FE" (ifo-Engineering + SFM)
# - USE_DYNAMIC_FI_PIPELINE = True   -> "Dynamic FI" (Top-N rolling FI Features)
# - Verwendet GlobalConfig(preset="debug") für schnelle Splits
# - Mini-HP-Grids, damit der Lauf sehr schnell ist
# =========================

import os, sys
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd

# --- 1) Pfad-Setup (wie in ET/EN/LGBM) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Imports aus dem Repo ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,    # expanding
    EWMA_CORR_SPEC,       # ewm (hier im Debug nicht nötig, aber import kompatibel)
    outputs_for_model,
)

from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)

# ⚠️ Wichtig: Du brauchst eine SFM-Wrapper-Klasse mit ET/EN/LGBM-kompatibler API:
# src/models/SFM.py -> class ForecastModel(params) mit .fit/.predict/.predict_one/.get_feature_importances()
from src.models.SFM import ForecastModel  # <- analog zu deinen anderen Modellen

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False  # False => Full FE; True => Dynamic FI
SEED = 42

MODEL_NAME = "sfm_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "sfm_debug"
outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten laden ---
y = load_target()             # ΔIP (DatetimeIndex)
X_ifo = load_ifo_features()   # ifo features (für Full FE)

FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print("Dynamic-FI Daten gefunden. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst die FI-Jobs laufen lassen (feature_importance.ipynb).")
        raise
else:
    # Full FE: Indizes alignen
    idx_common = y.index.intersection(X_ifo.index)
    y    = y.loc[idx_common]
    X_ifo = X_ifo.loc[idx_common]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Base-Config (DEBUG-Preset, schnelle Splits/Policies) ---
def base_cfg_debug() -> GlobalConfig:
    cfg = GlobalConfig(preset="debug")  # kurze Fenster/Splits für schnellen Lauf
    cfg.policy_window   = 6             # kleiner als 24 für schnellere Policy
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 2
    return cfg

cfg0 = base_cfg_debug()

# --- 6) Corr-Helper (kompatibel zu ET/EN/LGBM) ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 7) Mini-HP-Grids -------------------------------------------------
if USE_DYNAMIC_FI_PIPELINE:
    # --------- Dynamic FI (nur Modell-HPs + n_features_to_use) ----------
    print("Erstelle Mini-Grid für 'Dynamic FI'...")

    N_FEATURES_TO_USE = 10  # sehr klein, schneller Lauf

    # SFM-HPs (statisch, klein):
    # - n_factors: sehr kleine Auswahl
    n_factors_list = [2]          # minimal
    link_list      = ["ols"]      # reine OLS-Link (kein Ridge)
    ridge_lambda_list = [0.0]     # nur relevant falls link='ridge'

    weighting_opts = [{"sample_weight_decay": None}]  # weglassen für Speed

    def build_model_grid_dynamic_fi():
        grid = []
        for r, link, lam in product(n_factors_list, link_list, ridge_lambda_list):
            base = {
                "n_features_to_use": N_FEATURES_TO_USE,
                "n_factors": r,
                "link": link,              # 'ols' oder 'ridge'
                "ridge_lambda": lam,       # nur falls ridge
                "seed": SEED,
            }
            for w in weighting_opts:
                hp = dict(base)
                hp.update(w)
                grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # ----------------- Full FE (analog ET/EN/LGBM, aber ultra-klein) ---------------------
    print("Erstelle Mini-Grid für 'Full FE'...")

    # A) FE/DR-Listen (sehr klein; SFM macht intern Faktorextraktion → hier DR 'none')
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
    ]
    lag_candidates_list   = [(1, 2, 3)]   # sehr klein
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100]         # kleiner Screen
    redundancy_param_list = [0.90]
    dr_options_list       = [
        {"dr_method": "none"},            # SFM nutzt selbst PCs/Faktoren → hier keine zusätzliche DR
    ]

    # B) SFM-HPs (sehr klein)
    n_factors_list   = [2, 4]
    link_list        = ["ols"]      # nur OLS für Speed/Einfachheit
    ridge_lambda_list = [0.0]

    # C) Target Blocks & Weighting (aus für Speed/Einfachheit)
    target_block_options = [None]
    weight_opts          = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list,      # lags
            top_k_lags_list,          # k_lags
            use_rm3_list,             # rm3
            k1_topk_list,             # k1
            redundancy_param_list,    # red
            dr_options_list,          # dr_opt (dict)
        ]
        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                base_fe = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                }
                for r, link, lam in product(n_factors_list, link_list, ridge_lambda_list):
                    base_model = {
                        "n_factors": r,
                        "link": link,
                        "ridge_lambda": lam,
                        "seed": SEED,
                    }
                    for block_set in target_block_options:
                        for w in weight_opts:
                            hp = {
                                **base_fe,
                                **base_model,
                                "target_block_set": block_set,
                                **w,
                            }
                            hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("HP-Kombinationen (Mini):", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B Lauf (wie ET/EN/LGBM) ------------------------------------------
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        # Dynamic FI Lauf
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,   # Platzhalter (nicht genutzt im Dynamic-FI-Flow)
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        # Full FE Lauf (Debug)
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell sfm_debug wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)
Erstelle Mini-Grid für 'Full FE'...
HP-Kombinationen (Mini): 2
Erstes HP-Set: {'lag_candidates': (1, 2, 3), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'n_factors': 2, 'link': 'ols', 'ridge_lambda': 0.0, 'seed': 42, 'target_block_set': None, 'sample_weight_decay': None}
[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=48, OOS=49-60 | configs=2
  - Config 1/2
    · Month 5/12 processed | running...RMSE=1.1859
    · Month 10/12 processed | running...RMSE=1.3636
    · Month 12/12 processed | running...RMSE=1.4487
  - Config 2/2
    · Month 5/12 processed | running...RMSE=0.8442
    · Month 10/12 processed